In [ ]:
#from model import CVAE
from utils import *
import numpy as np
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
import time
import argparse
from rdkit import Chem
from rdkit.Chem.Descriptors import ExactMolWt
from rdkit.Chem.Crippen import MolLogP
from rdkit.Chem.rdMolDescriptors import CalcTPSA

In [ ]:
batch_size = 34
iteration =10
latent_size =300
unit_size =512
n_rnn_layer=3
seq_length=185
mean=0.0
stddev=1.0
num_prop=329
save_file = "NewSmilesWeirdLoss300LS300Epochs/model_300.ckpt-300"
target_prop = "3000"
prop_file = "smiles_new_dti.txt"
result_filename  = "result0.8Loss300LS.txt"
lr=0.0001


In [ ]:
_, _, char, vocab, _, _ = load_data(prop_file,seq_length)
vocab_size = len(char)

In [ ]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tensorflow as tf2 
import tensorflow_addons as tfa
import numpy as np
import threading
from utils import *
from rdkit import Chem
from rdkit.Chem.Descriptors import ExactMolWt
from rdkit.Chem.Crippen import MolLogP
from rdkit.Chem.rdMolDescriptors import CalcTPSA


class CVAE2():
    def __init__(self,
                 vocab_size,
                 char,
                 b_s,
                 l_s,
                 l_r,
                 num_prop,
                 stddev,
                 mean,
                 u_s,
                 n_rnn_layer
                  ):
        self.char = char
        self.vocab_size = vocab_size
        self.batch_size = b_s
        self.latent_size = l_s
        self.lr = tf.Variable(l_r, trainable=False)
        self.num_prop = num_prop
        self.stddev = stddev
        self.mean = mean
        self.unit_size = u_s
        self.n_rnn_layer = n_rnn_layer
        
        self._create_network()


    def _create_network(self):
        self.X = tf.placeholder(tf.int32, [self.batch_size, None])
        self.Y = tf.placeholder(tf.int32, [self.batch_size, None])
        self.C = tf.placeholder(tf.float32, [self.batch_size, self.num_prop])
        self.L = tf.placeholder(tf.int32, [self.batch_size])
        

        
        decoded_rnn_size = [self.unit_size for i in range(self.n_rnn_layer)]
        encoded_rnn_size = [self.unit_size for i in range(self.n_rnn_layer)]
        
        with tf.variable_scope('decode'):
            decode_cell=[]
            for i in decoded_rnn_size[:]:
                decode_cell.append(tf.nn.rnn_cell.LSTMCell(i))
            self.decoder = tf.nn.rnn_cell.MultiRNNCell(decode_cell)
        
        with tf.variable_scope('encode'):
            encode_cell=[]
            for i in encoded_rnn_size[:]:
                encode_cell.append(tf.nn.rnn_cell.LSTMCell(i))
            self.encoder = tf.nn.rnn_cell.MultiRNNCell(encode_cell)
        
        self.encoder_cell = tf.nn.rnn_cell.LSTMCell(self.unit_size)

        self.weights = {}
        self.biases = {}
        self.eps = {
            'eps' : tf.random_normal([self.batch_size, self.latent_size], stddev=self.stddev, mean=self.mean)
        }


        self.weights['softmax'] = tf.get_variable("softmaxw", initializer=tf.random_uniform(shape=[decoded_rnn_size[-1], self.vocab_size], minval = -0.1, maxval = 0.1))       
        
        self.biases['softmax'] =  tf.get_variable("softmaxb", initializer=tf.zeros(shape=[self.vocab_size]))
        self.weights['out_mean'] = tf.get_variable("outmeanw", initializer=tf2.initializers.GlorotUniform(), shape=[self.unit_size, self.latent_size]),
        self.weights['out_log_sigma'] = tf.get_variable("outlogsigmaw", initializer=tf2.initializers.GlorotUniform(), shape=[self.unit_size, self.latent_size]),
        self.biases['out_mean'] = tf.get_variable("outmeanb", initializer=tf2.initializers.GlorotUniform(), shape=[self.latent_size]),
        self.biases['out_log_sigma'] = tf.get_variable("outlogsigmab", initializer=tf2.initializers.GlorotUniform(), shape=[self.latent_size]),

        self.embedding_encode = tf.get_variable(name = 'encode_embedding', shape = [self.latent_size, self.vocab_size], initializer = tf.random_uniform_initializer( minval = -0.1, maxval = 0.1))
        
        self.embedding = tf.nn.embedding_lookup(self.embedding_encode, self.X)

        
        self.latent_vector, self.mean, self.log_sigma = self.encode()
        
        self.decoded, decoded_logits = self.decode(self.latent_vector)
        self.mol_pred = tf.argmax(self.decoded, axis=2)

        #self.Y_generated = self.generate()

        weights = tf.sequence_mask(self.L, tf.shape(self.X)[1])
        weights = tf.cast(weights, tf.int32)
        weights = tf.cast(weights, tf.float32)

        self.reconstr_loss = tf.reduce_mean(tfa.seq2seq.sequence_loss(
            logits=decoded_logits, targets=self.Y, weights=weights))
        self.latent_loss = self.cal_latent_loss(self.mean, self.log_sigma)
        self.real_loss =  tf.py_func(self.convert_to_smiles_tf, inp=[self.mol_pred, list(self.char.keys()), self.reconstr_loss, self.latent_loss, self.Y], Tout=tf.float32)
        #npa = tf.make_ndarray(real_molecule)
        
       

        # Loss

        self.loss = self.latent_loss + self.reconstr_loss + self.real_loss
        #self.loss = self.reconstr_loss 
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.opt = optimizer.minimize(self.loss)
        
        self.sess = tf.Session()
        
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
        
        self.saver = tf.train.Saver(max_to_keep=None)
        #tf.train.start_queue_runners(sess=self.sess)


        print ("Network Ready")

    def encode(self): 
        X = tf.nn.embedding_lookup(self.embedding_encode, self.X)
        C = tf.expand_dims(self.C, 1)
        C = tf.tile(C, [1, tf.shape(X)[1], 1])
        inp = tf.concat([X, C], axis=-1)

        
        _, state = tf.nn.dynamic_rnn(self.encoder, inp, dtype=tf.float32, scope = 'encode', sequence_length = self.L)
        c,h = state[-1]
        self.weights['out_mean'] = tf.reshape(self.weights['out_mean'], [self.unit_size, -1])
        self.weights['out_log_sigma'] = tf.reshape(self.weights['out_log_sigma'], [self.unit_size, -1])
        mean = tf.matmul(h, self.weights['out_mean'])+self.biases['out_mean']
        log_sigma = tf.matmul(h, self.weights['out_log_sigma'])+self.biases['out_log_sigma']
        

        
        retval = mean+tf.exp(log_sigma/2.0)*self.eps['eps']
        print("latent space shape", tf.shape(retval))
        return retval, mean, log_sigma

    def decode(self, Z):
        seq_length=tf.shape(self.X)[1]
        print("seq length", seq_length)
        new_Z = tf.tile(tf.expand_dims(Z, 1), [1, seq_length, 1])
        C = tf.expand_dims(self.C, 1)
        C = tf.tile(C, [1, tf.shape(self.X)[1], 1])
        X = tf.nn.embedding_lookup(self.embedding_encode, self.X)
        print("X",  tf.shape(X))
        inputs = tf.concat([new_Z, X, C], axis=-1)
        print("inputs", tf.shape(inputs))
        
        self.initial_decoded_state = tuple([tf.nn.rnn_cell.LSTMStateTuple(tf.zeros((self.batch_size, self.unit_size)), tf.zeros((self.batch_size, self.unit_size))) for i in range(self.n_rnn_layer)])
        #self.initial_decoded_state=self.decoder.zero_state() 
        Y, self.output_decoded_state = tf.nn.dynamic_rnn(self.decoder, inputs, dtype=tf.float32, scope = 'decode', sequence_length = self.L, initial_state=self.initial_decoded_state)

        Y = tf.reshape(Y, [self.batch_size*seq_length, -1])
        Y = tf.matmul(Y, self.weights['softmax'])+self.biases['softmax']
        Y_logits = tf.reshape(Y, [self.batch_size, seq_length, -1])
        Y = tf.nn.softmax(Y_logits)
        print("Y:", Y, "and logits:", Y_logits)
        return Y, Y_logits

    def save(self, ckpt_path, global_step):
        self.saver.save(self.sess, ckpt_path, global_step = global_step)
        #print("model saved to '%s'" % (ckpt_path))

    def assign_lr(self, learning_rate):
        self.sess.run(tf.assign(self.lr, learning_rate ))
    
    def restore(self, ckpt_path):
        self.saver.restore(self.sess, ckpt_path)

    def get_latent_vector(self, x, c, l):
        return self.sess.run(self.latent_vector, feed_dict={self.X : x, self.C : c, self.L : l})

    def cal_latent_loss(self, mean, log_sigma):
        latent_loss = tf.reduce_mean(-0.5*(1+log_sigma-tf.square(mean)-tf.exp(log_sigma)))
        return latent_loss
    
    def train(self, x, y, l, c):
        _, loss, embedding = self.sess.run([self.opt, self.loss, self.embedding], feed_dict = {self.X :x, self.Y:y, self.L : l, self.C : c})
        return loss, embedding
    
    def test(self, x, y, l, c):
        mol_pred, loss  = self.sess.run([self.mol_pred, self.loss], feed_dict = {self.X :x, self.Y:y, self.L : l, self.C : c})
        return loss

    
    def convert_to_smiles_tf(self, vector, char, re_loss, la_loss, target):
        list_char = char
        #print(list_char)
        #list_char = char.tolist()
        
        vector = vector.astype(int)
        errors = []
        new_string = ""

        for molecule in range(len(vector)):
            drug = []
            y = []
            for letter in range(len(vector[molecule])):
                drug.append(list_char[vector[molecule][letter]].decode('utf-8'))
                y.append(list_char[target[molecule][letter]].decode('utf-8'))

            drugS = "".join(drug)
            y = "".join(y)
            #print("FINAL DRUG", drugS)
            print("FINAL DRUG", drugS)
            print("ACTUAL DRUG", y)
            index = drugS.find('E')
            if(index == -1):
                index = 186
            real_molecule = Chem.MolFromSmiles(drugS[0:index])
            real_loss = 0.8
            if real_molecule is not None:
                real_loss = 0.0
            errors.append(real_loss)
        print("Reconstruction loss", re_loss)
        print("Latent loss", la_loss)
        print("Real loss", np.mean(errors) + 0.035*self.numIncBrackets('(', ')', drugS[0:index]) + 0.035*self.numWrongRing(drugS[0:index]))
        return np.float32(np.mean(errors) + 0.035*self.numIncBrackets('(', ')', drugS[0:index]) + 0.035*self.numWrongRing(drugS[0:index]))

    def numIncBrackets(self, charS, charE, string):
        openC= 0
        count =0

        for i in range(len(string)):
            if string[i]==charS:
                openC += 1
            elif string[i]==charE:
                openC -= 1
            if(openC<0):
                count += 1
                openC += 1
        return count+openC
    
    def numWrongRing(self, string):
        sum = 0
        for number in range(20):
            count= 0
            for i in range(len(string)):
                if string[i]==number:
                    count += 1
            if(count != 0 and count==1):
                sum += (count%2)
            elif(count != 0 and count > 1):
                sum += (count%2 + (count - 2))
        return sum
    


    def sample(self, latent_vector, c, start_codon, seq_length):
        l = np.ones((self.batch_size)).astype(np.int32)
        x=start_codon
        preds = []
        for i in range(seq_length):
            if i==0:
                x, state = self.sess.run([self.mol_pred, self.output_decoded_state], feed_dict = {self.X:x, self.latent_vector:latent_vector, self.L : l, self.C : c})
            else:
                x, state = self.sess.run([self.mol_pred, self.output_decoded_state], feed_dict = {self.X:x, self.latent_vector:latent_vector, self.L : l, self.C : c, self.initial_decoded_state:state})
            preds.append(x)
        return np.concatenate(preds,1).astype(int).squeeze()


In [ ]:
model = CVAE2(vocab_size, vocab, batch_size, latent_size, lr, num_prop, stddev, mean, unit_size,n_rnn_layer)
model.restore(save_file)

In [ ]:
print ('Number of parameters : ', np.sum([np.prod(v.shape) for v in tf.trainable_variables()]))

In [ ]:
#specifying the condition vector; which is the drug target interaction profile showing which proteins you want the drug to bid to 
target_dtis = np.array([[float(p) for p in "0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	1	1	1	0	1	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0".split()] for _ in range(batch_size)])
print(target_prop)
start_codon = np.array([np.array(list(map(vocab.get, 'X')))for _ in range(batch_size)])
#print(start_codon)

In [ ]:
smiles = []
for _ in range(iteration):
    latent_vector = s = np.random.normal(mean, stddev, (batch_size, latent_size))
    generated = model.sample(latent_vector, target_dtis, start_codon, seq_length)
    smiles += [convert_to_smiles(generated[i], char) for i in range(len(generated))]

In [ ]:
def applytab(row):
    return '\t'.join(map(str,row.values))
#print('\t'.join(map(str,df.columns))) # to print the column names if requiredstr =


In [ ]:
print ('number of trial : ', len(smiles))
smiles = list(set([s.split('E')[0] for s in smiles]    ))
print ('number of generate smiles (after remove duplicated ones) : ', len(smiles))
ms = [Chem.MolFromSmiles(s) for s in smiles]
ms2 = [m for m in ms if m is not None]
print ('number of valid smiles : ', len(ms2))

In [ ]:
result_filename  = "results.txt"

with open(result_filename, 'w') as w:
    
    w.write('smiles\tMW\tLogP\tTPSA\n')
    for m in ms:
        #print(m)
        if m is not None:
            w.write(str(m)+"\n")
    w.close()           